In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp importers.gmail

In [ ]:
# export
import imaplib, email
from integrators.data.schema import Account, EmailMessage, MessageChannel
from integrators.pod.client import PodClient
from email import policy
from email.utils import getaddresses
from integrators.imports import *

In [ ]:
x = Path("~")

In [ ]:
Path.home()

PosixPath('/Users/koen')

In [ ]:
x.resolve()

PosixPath('/Users/koen/workspace/pyintegrators/nbs/~')

# Email importer

This is an email importer, it imports your emails from an email server using SMTP. It was tested on gmail, but should work for most SMTP-servers. _The recommended usage for Gmail is to enable two-factor authentication. In this case, make sure you allow [SMTP-connections](https://www.gmass.co/blog/gmail-smtp/) and have enabled an application password (explained in the same link):_

## ImapClient

This importer communicates with email providers over imap. We created a convenience class around pythons imaplib that lets you list your mailboxes, retriev your mails and get their content.

In [ ]:
# export
DEFAULT_GMAIL_HOST = 'imap.gmail.com'
DEFAULT_GMAIL_INBOX = '"[Gmail]/All Mail"' # Note the double quotes here
DEFAULT_PORT = 993

class IMAPClient():
    
    def __init__(self, username, app_pw, host=DEFAULT_GMAIL_HOST, port=DEFAULT_PORT, inbox=DEFAULT_GMAIL_INBOX):   
        assert username is not None and app_pw is not None
        self.client = imaplib.IMAP4_SSL(host, port=port)
        self.client.login(username, app_pw)
        self.client.select(inbox) # connect to inbox.
        
    def list_mailboxes(self):
        return self.client.list()
    
    def get_all_mail_uids(self):
        result, data = self.client.uid('search', None, "ALL") # search and return uids instead
        return data[0].split()
    
    def get_mail(self, uid):
        if self.client.host == DEFAULT_GMAIL_HOST:
            # Use Google's threading method, in which every thread has an ID
            result, (data, _) = self.client.uid('fetch', uid, '(RFC822 X-GM-THRID)')
            thread_id = data[0].decode("utf-8").split(" ")[2]
            raw_email = data[1]
            return (raw_email, thread_id)
        else:
            # Threading not yet implemented for IMAP threading
            result, (data, _) = self.client.uid('fetch', uid, '(RFC822)')
            raw_email = data[1]
            return (raw_email, None)

def part_to_str(part):
    bytes_ = part.get_payload(decode=True)
    charset = part.get_content_charset('iso-8859-1')
    chars = bytes_.decode(charset, 'replace')
    return chars

def _get_all_parts(part):
    payload = part.get_payload()
    if isinstance(payload, list):
        return [x for p in payload for x in _get_all_parts(p)]
    else:
        return [part]

## Utils

In [ ]:
# export

# TODO: should probably become a general utility function
def get_unique_accounts(all_mails):
    all_accounts = {}
    for email_item in all_mails:
        for edge in email_item.get_all_edges():
            account = edge.traverse(email_item)
            if not account.externalId in all_accounts:
                all_accounts[account.externalId] = account

    for email_item in all_mails:
        for edge in email_item.get_all_edges():
            edge.target = all_accounts[edge.target.externalId]
            
    return list(all_accounts.values())

# TODO: should probably become a general utility function
def get_g_attr(item, name, data_type, default_value=None):
    first_or_default = next((att for att in item.genericAttribute if att.name == name), None)
    if first_or_default == None:
        return default_value
    else:
        if data_type == 'int':
            return first_or_default.intValue
        elif data_type == 'bool':
            return first_or_default.boolValue
        elif data_type == 'float':
            return first_or_default.floatValue
        elif data_type == 'string':
            return first_or_default.stringValue
        elif data_type == 'datetime':
            return first_or_default.stringValue
        else:
            raise Exception(f"datatype {data_type} is not supported")

## EmailImporter

In [ ]:
# export

from integrators.data.schema import *
from integrators.imports import *
from integrators.indexers.indexer import ImporterBase, test_registration

class GmailImporter(ImporterBase):        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.imap_client = None
       
    def get_data(self, client, indexer_run):
        print('this function is a workaround (this Importer is an Indexer temporarily)')
        
    def set_imap_client(self, importer_run):
        imap_host = get_g_attr(importer_run, 'host', 'string', DEFAULT_GMAIL_HOST)
        port = get_g_attr(importer_run, 'port', 'int', DEFAULT_PORT)
        assert imap_host is not None and port is not None
        print(f'Using, HOST: {imap_host}, PORT: {port}')
        
        self.imap_client = IMAPClient(username=importer_run.username, 
                                     app_pw=importer_run.password, 
                                     host=imap_host,
                                     port=993)
    
    @staticmethod
    def get_timestamp_from_message(message):
        date = message["date"]
        parsed_time = email.utils.parsedate(date)
        dt = email.utils.parsedate_to_datetime(date)
        timestamp = int(dt.timestamp() * 1000)

        return timestamp
    
    @staticmethod
    def get_accounts(message, field):
        addresses = getaddresses(message.get_all(field, []))
        return [Account(externalId=address) for name, address in addresses]
    
    @staticmethod
    def get_content(message):
        maintype = message.get_content_maintype()
        if maintype == 'multipart':

            parts = _get_all_parts(message)
            res = None
            html_parts = [part_to_str(part) for part in parts if part.get_content_type() == "text/html"]
            if len(html_parts) > 0:
                if len(html_parts) > 1:
                    error_msg = "\n AND \n".join(html_parts)
                    print(f"WARNING: FOUND MULTIPLE HTML PARTS IN ONE MESSAGE {error_msg}")
                return html_parts[0]
            else:
                return parts[0].get_payload()

        elif maintype == 'text':
            return message.get_payload()
        
    @staticmethod    
    def get_attachments(message): return list(message.iter_attachments())
        
    def create_item_from_mail(self, mail, thread_id=None):
        """Creates a schema-item from an existing mail"""
        message = email.message_from_bytes(mail, policy=policy.SMTP)

        message_id, subject = message["message-id"], message["subject"]
        timestamp = self.get_timestamp_from_message(message)

        content = self.get_content(message)
        attachments = self.get_attachments(message)   

        email_item = EmailMessage(externalId=message_id, subject=subject, dateSent=timestamp, content=content)

        for a in self.get_accounts(message, 'from'): email_item.add_edge('sender', a)
        for a in self.get_accounts(message, 'to'): email_item.add_edge('receiver', a)
        for a in self.get_accounts(message, 'reply-to'): email_item.add_edge('replyTo', a)

        if thread_id != None:
            email_item.add_edge('messageChannel', MessageChannel(externalId=thread_id))

        return email_item

    def get_mails(self, gmail_ids, stop_at=1e9):
        all_mails = []

        # Download files
        for i, gmail_id in enumerate(gmail_ids):
            if i >= stop_at:
                print(f"stopped early at {stop_at}")
                break

            mail, thread_id = self.imap_client.get_mail(gmail_id)

            item = self.create_item_from_mail(mail, thread_id=thread_id)
            all_mails.append(item)

        return all_mails
        
    def run(self, importer_run, pod_client=None):    
        self.set_imap_client(importer_run)
        
        stop_early_at = get_g_attr(importer_run, 'max_number', 'int', 10)
        print(f'MAX_ITEMS: {stop_early_at}')
        
        gmail_ids = self.imap_client.get_all_mail_uids()
        all_mails = self.get_mails(gmail_ids, stop_early_at)

        # TODO: create better way to do this
        all_accounts = get_unique_accounts(all_mails)

        # Create all email and account items
        for item in all_mails + all_accounts: pod_client.create(item)

        # Create all edges from emails to accounts/messageThreads
        for email_item in all_mails: pod_client.create_edges(email_item.get_all_edges())
        print(f"Finished running {self}")

The email importer has the following parameters

- **username** Your email address
- **password** Your email password. In case you're using gmail, use your application password
- _generic attributes_
- **host** The URL of the host (defaults to imap.gmail.com)
- **port** The port of the server (defaults to 993 for gmail)
- **max_number** Max number of emails to download. Leave unset for unlimited

## Tests

### Download all mails from your account

In [ ]:
# %nbdev_slow_test
# This cell is meant to be able to test the importer locally

# Store your credentials in this file:
file = open('/tmp/credentials_gmail.txt','r')
imap_user = file.readline().strip('\n')
imap_pw = file.readline().strip('\n')

pod_client = PodClient()
pod_client.delete_all()

importer_run = ImporterRun.from_data(progress=0, username=imap_user, password=imap_pw)
importer = Importer.from_data()
host_item = GenericAttribute(name='host', stringValue=DEFAULT_GMAIL_HOST)
port_item = GenericAttribute(name='port', intValue=993)
max_number_item = GenericAttribute(name='max_number', intValue=10)
importer_run.add_edge('genericAttribute', host_item)
importer_run.add_edge('genericAttribute', port_item)
importer_run.add_edge('genericAttribute', max_number_item)
importer_run.add_edge('importer', importer)

importer_temp = GmailImporter.from_data()
importer_temp.run(importer_run=importer_run, pod_client=pod_client)

Using, HOST: imap.gmail.com, PORT: 993
MAX_ITEMS: 10
stopped early at 10


### Parsing

In [ ]:
test = b"""\
Message-id: 1234\r
From: user1 <a@gmail.com>\r
To: user1 <b@gmail.com>\r
Reply-to: user1 <c@gmail.com>\r
Subject: the subject\r
Date: Mon, 04 May 2020 00:37:44 -0700\r

This is content"""

gmail_importer = GmailImporter()
mail_item = gmail_importer.create_item_from_mail(test, 'message_channel_id')

assert mail_item.externalId == '1234'
assert mail_item.sender[0].externalId == 'a@gmail.com'
assert mail_item.receiver[0].externalId == 'b@gmail.com'
assert mail_item.replyTo[0].externalId == 'c@gmail.com'
assert mail_item.subject == 'the subject'
assert mail_item.content == 'This is content'
assert mail_item.dateSent == gmail_importer.get_timestamp_from_message(email.message_from_bytes(test))
assert mail_item.messageChannel[0].externalId == 'message_channel_id'

### Attachments

In [ ]:
# Test attachment parsing (basic support)
gmail_importer = GmailImporter()
message = email.message.EmailMessage()
message.set_content('aa')
message.add_attachment(b'bb', maintype='image', subtype='jpeg', filename='sample.jpg')
message.add_attachment(b'cc', maintype='image', subtype='jpeg', filename='sample2.jpg')
content = gmail_importer.get_content(message)
attachments = gmail_importer.get_attachments(message)

assert content == 'aa\n'
assert attachments[0].get_content() == b'bb'
assert attachments[1].get_content() == b'cc'

### Calling the importer from the pod

In [ ]:
%nbdev_slow_test

# This cell is meant to be able to call the importer locally (simulating the front-end)

pod_client = PodClient(url='http://0.0.0.0:3030')
pod_client.create(importer_run)
pod_client.create(importer)
pod_client.create(host_item)
pod_client.create(port_item)
pod_client.create(max_number_item)
pod_client.create_edges(importer_run.get_all_edges())

json = {
            'databaseKey':pod_client.database_key,
            'payload':{
                 'uid':importer_run.uid,
                 'servicePayload': {
                     'databaseKey':pod_client.database_key,
                     'ownerKey':pod_client.owner_key
                 }
            }
       }
print(importer_run.uid)
print(requests.post(f'http://0.0.0.0:3030/v2/{pod_client.owner_key}/run_importer',
                   json=json).content)

UsageError: Line magic function `%nbdev_slow_test` not found.
